# this is Pixegami tutorial
### from this video :https://www.youtube.com/watch?v=tcqEUSNCn8I

In [50]:
from dotenv import load_dotenv
import os
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()
directory_path='/Users/matansharon/python/chat_with_docs/data/text'


In [55]:
def load_and_split_documents():
    loader = DirectoryLoader(directory_path)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True
    )
    chunks=text_splitter.split_documents(documents)
    return chunks
def create_db(chunks):
    path='chroma_db'
    if not os.path.exists(path):
        
        db=Chroma.from_documents(documents=chunks,embedding=OpenAIEmbeddings(),persist_directory=path)
        return db
    return load_db()
def load_db():
    db = Chroma(persist_directory="chroma_db",embedding_function=OpenAIEmbeddings())
    return db
def get_results_with_scores(query,db):
    bar=0.5
    res=db.similarity_search_with_relevance_scores(query,k=3)
    if len(res)==0 or res[0][1]<bar:
        return 'No results found'
    r=[]
    for i in range(len(res)):
        if res[i][1]>bar:
            r.append(res[i])
    return r
def get_prompt_template(results,query):
    template="""
    answer the question base only on the following context:
    {context}
    answer the question base on the above context: {query}
    
    """
    context_text="\n\n---\n\n".join([chunk.page_content for chunk,_score in results])
    prompt_tamplate=ChatPromptTemplate.from_template(template)
    res=prompt_tamplate.format(context=context_text,query=query)
    return res
def main():
    chunks=load_and_split_documents()
    db=create_db(chunks)
    query=input('Enter your query: ')
    results=get_results_with_scores(query,db)
    prompt_template=get_prompt_template(results,query)
    
    model=ChatOpenAI()
    response=model.predict(prompt_template)
    print(response)

    


In [56]:
main()


QLORA is an efficient finetuning approach that reduces memory usage and allows for finetuning of large language models on a single GPU. It backpropagates gradients through a frozen, 4-bit quantized pretrained language model into Low Rank Adapters (LoRA) and introduces innovations such as a new data type called 4-bit NormalFloat (NF4) to save memory without sacrificing performance. The best model family resulting from QLORA, named Guanaco, outperforms previous models on the Vicuna benchmark and achieves 99.3% of the performance level of ChatGPT with only 24 hours of finetuning on a single GPU.
